# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [39]:
# All import statements needed for the project, for example:
import json
import pathlib
import urllib.parse

import pandas as pd
import geopandas as gpd
import requests
import re
import psycopg2
import shapely

from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, TIMESTAMP
from geoalchemy2 import Geometry
from datetime import datetime, timedelta

In [14]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "NJflFVV2YiwlXmMlt4Y9jwTGO"
API_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
API_TREE='https://data.cityofnewyork.us/resource/uvpi-gqnh.json'

## Soql for NYC_311 and NYC_TREE Download
SOQL_BASE_QUERY_311 = f"{API_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000"
SOQL_QUERIES_311 = {f"query_{i}": f"{SOQL_BASE_QUERY_311}&$offset={(i-1)*1000000}" for i in range(1, 26)}
SOQL_QUERY_TREE=f"{API_TREE}?$where=created_at between '2015-01-01T00:00:00' and '2015-12-31T23:59:59'&$limit=1000000000"

DB_NAME = "GroupProject"
DB_USER = 'postgres'
DB_PASSWORD = ''
DB_HOST = 'localhost'
DB_URL=f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'
DB_SCHEMA_FILE = "schema.sql"

# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [15]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

### Download and Clean NYC_311

In [16]:
def download_nyc_json_data(query):
    headers = {'X-App-Token': NYC_DATA_APP_TOKEN }
    response = requests.get(query, headers=headers)
    df = pd.read_json(response.text)
    return df

In [17]:
def download_311_data(queries_dict):
    for query_name, query in queries_dict.items():
        query_df = download_nyc_json_data(query)
        query_df.to_csv(f"{query_name}_data.csv", index=False)

In [19]:
def load_and_clean_311_data():
    all_data = pd.concat([pd.read_csv(f"{query_name}_data.csv", low_memory=False) for query_name in SOQL_QUERIES_311], ignore_index=True)
    
    # Filter for noise-related complaints
    noise_dataset = all_data[all_data['complaint_type'].str.lower().str.contains("noise")]

    # Select relevant columns
    noise_dataset = noise_dataset[['unique_key', 'created_date', 'complaint_type', 'incident_zip', 'latitude', 'longitude']]

    # Convert latitude and longitude to a GeoDataFrame with a 'location' column
    noise_dataset['location'] = gpd.points_from_xy(noise_dataset['longitude'], noise_dataset['latitude'], crs="EPSG:4326")

    # Drop latitude and longitude columns
    noise_dataset = noise_dataset.drop(['latitude', 'longitude'], axis=1)

    # Rename columns
    noise_dataset = noise_dataset.rename(columns={"incident_zip": "zipcode"})

    # Drop rows with missing zip codes
    noise_dataset = noise_dataset.dropna(subset=['zipcode'])

    # Change data types
    convert_dict = {'unique_key': int, 'created_date': 'datetime64[ns]', 'complaint_type': str, 'zipcode': int}
    noise_dataset = noise_dataset.astype(convert_dict)

    return gpd.GeoDataFrame(noise_dataset, geometry='location')

### Download and Clean NYC_tree

In [20]:
def download_and_clean_tree_data():
    # Download tree data for the year 2015
    tree_data = download_nyc_json_data(SOQL_QUERY_TREE)

    # Select relevant columns
    selected_columns = ['tree_id', 'status', 'health', 'spc_common', 'zipcode', 'latitude', 'longitude']
    tree_data = tree_data[selected_columns]

    # Convert latitude and longitude to a GeoDataFrame with a 'location' column
    tree_data['location'] = gpd.points_from_xy(tree_data['longitude'], tree_data['latitude'], crs="EPSG:4326")

    # Drop latitude and longitude columns
    tree_data = tree_data.drop(['latitude', 'longitude'], axis=1)

    # Rename columns
    tree_data = tree_data.rename(columns={"spc_common": "species"})

    return gpd.GeoDataFrame(tree_data, geometry='location')

### Load and Clean NYC_Zipcode

In [21]:
def load_and_clean_zipcodes_data(zipcode_datafile): 
    try:
        data = gpd.read_file(zipcode_datafile)
        
        # Select relevant columns
        selected_columns = ['ZIPCODE', 'geometry']
        data_nyc = data[selected_columns]
        
        # Rename columns for clarity
        zipcode_nyc = gpd.GeoDataFrame(data_nyc, geometry='geometry').rename(
            columns={"ZIPCODE": "zipcode"})
        
        # Change data types and coordinate reference system
        zipcode_nyc = zipcode_nyc.to_crs(epsg=4326)
        zipcode_nyc = zipcode_nyc.astype({'zipcode': int})
        
        # Drop duplicate zipcodes, keeping the first occurrence
        zipcode_nyc = zipcode_nyc.drop_duplicates(subset=['zipcode'])
        
        return zipcode_nyc
    
    except Exception as e:
        # Handle exceptions
        print(f"Error: {e}. Could not read file: {zipcode_datafile}")
        return None

### Load and Clean NYC_Zillow

In [22]:
def load_and_clean_zillow_data(zillow_datafile): 
    data = pd.read_csv(zillow_datafile)
    
    # Only get the data of "NY" state
    zillow_data_NY = data[data.City == "New York"]
    
    # Get DataFrame containing selected columns and all numeric columns
    numeric_columns = data.filter(regex='^\d').columns.tolist()
    zillow_NY_dataset = zillow_data_NY[["RegionName"] + numeric_columns]
    
    # Rename some columns
    zillow_NY_dataset = zillow_NY_dataset.rename(columns={"RegionName": "zipcode"})
    zillow_NY_dataset.columns = ["zipcode"] + [col.replace('-', '') for col in numeric_columns]
    
    return zillow_NY_dataset

### Load all data

In [23]:
def load_all_data():
    data_311 = load_and_clean_311_data()
    data_tree = download_and_clean_tree_data()
    data_zipcode = load_and_clean_zipcodes_data(ZIPCODE_DATA_FILE)
    data_zillow = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
    
    return (
        data_311,
        data_tree,
        data_zipcode,
        data_zillow
    )

In [24]:
data_311, data_tree, data_zipcode, data_zillow = load_all_data()

In [25]:
# Show basic info about each dataframe
data_311.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 5056549 entries, 9 to 24626307
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   unique_key      int64         
 1   created_date    datetime64[ns]
 2   complaint_type  object        
 3   zipcode         int64         
 4   location        geometry      
dtypes: datetime64[ns](1), geometry(1), int64(2), object(1)
memory usage: 231.5+ MB


In [26]:
# Show first 5 entries about each dataframe
data_311.head()

,unique_key,created_date,complaint_type,zipcode,location
9,59273608,2023-10-30 23:58:22,Noise - Residential,11233,POINT (-73.93110 40.68470)
10,59277713,2023-10-30 23:58:20,Noise - Street/Sidewalk,11229,POINT (-73.95575 40.60559)
17,59271083,2023-10-30 23:54:45,Noise - Street/Sidewalk,10025,POINT (-73.96521 40.80111)
18,59270832,2023-10-30 23:54:19,Noise - Residential,11102,POINT (-73.92473 40.76758)
20,59272404,2023-10-30 23:54:04,Noise - Vehicle,11434,POINT (-73.77976 40.69105)


In [27]:
data_tree.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 509525 entries, 0 to 509524
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   tree_id   509525 non-null  int64   
 1   status    509525 non-null  object  
 2   health    485498 non-null  object  
 3   species   485498 non-null  object  
 4   zipcode   509525 non-null  int64   
 5   location  509525 non-null  geometry
dtypes: geometry(1), int64(2), object(3)
memory usage: 23.3+ MB


In [28]:
data_tree.head()

,tree_id,status,health,species,zipcode,location
0,180683,Alive,Fair,red maple,11375,POINT (-73.84422 40.72309)
1,200540,Alive,Fair,pin oak,11357,POINT (-73.81868 40.79411)
2,204026,Alive,Good,honeylocust,11211,POINT (-73.93661 40.71758)
3,204337,Alive,Good,honeylocust,11211,POINT (-73.93446 40.71354)
4,189565,Alive,Good,American linden,11215,POINT (-73.97598 40.66678)


In [29]:
data_zipcode.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 248 entries, 0 to 262
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   zipcode   248 non-null    int64   
 1   geometry  248 non-null    geometry
dtypes: geometry(1), int64(1)
memory usage: 5.8 KB


In [30]:
data_zipcode.head()

,zipcode,geometry
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [31]:
data_zillow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 4 to 6721
Columns: 106 entries, zipcode to 20230930
dtypes: float64(105), int64(1)
memory usage: 121.2 KB


In [32]:
data_zillow.head()

,zipcode,20150131,20150228,20150331,20150430,20150531,20150630,20150731,20150831,20150930,...,20221231,20230131,20230228,20230331,20230430,20230531,20230630,20230731,20230831,20230930
4,11385,NaN,2087.527084,NaN,2149.924252,2166.263698,2148.992886,2190.098591,2264.966715,2297.900917,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783
6,11208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2508.670432,2588.030194,2613.790654,2585.561351,2633.200754,2672.038493,2806.918757,2765.224364,2737.547470,2728.733333
12,11236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2285.460026,2362.500000
13,10467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2145.642295,2155.617718,2172.346611,2160.962748,2110.533203,2180.323655,2276.372290,2334.204728,2353.686402,2423.888889
14,11373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2199.459063,2255.604528,2262.101623,2271.514956,2250.182334,2231.959479,2257.413993,2247.592851,2302.557354,2292.994444


## Part 2: Storing Data

In [33]:
## first step: create database 'GroupProject'in PostgreSQL
!createdb GroupProject

createdb: error: database creation failed: ERROR:  database "GroupProject" already exists


In [34]:
!psql --dbname GroupProject -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [65]:
def create_tables(engine):
    metadata = MetaData()

    nyc_311 = Table(
        'nyc_311', metadata,
        Column('unique_key', Integer, primary_key=True),
        Column('created_date', TIMESTAMP),
        Column('complaint_type', String),
        Column('zipcode', Integer),
        Column('location', Geometry('POINT', srid=4326))
    )

    nyc_tree = Table(
        'nyc_tree', metadata,
        Column('tree_id', Integer, primary_key=True),
        Column('status', String),
        Column('health', String),
        Column('species', String),
        Column('zipcode', Integer),
        Column('location', Geometry('POINT', srid=4326))
    )

    nyc_zipcode = Table(
        'nyc_zipcode', metadata,
        Column('zipcode', Integer, primary_key=True),
        Column('location', Geometry('POLYGON', srid=4326))
    )
    
    nyc_zillow = Table(
        'nyc_zillow', metadata,
        Column('zipcode', Integer, primary_key=True)
    )

    data_columns = data_zillow.columns[1:]
    for data_column in data_columns:
        nyc_zillow.append_column(Column(data_column, Float))

    metadata.create_all(engine)

In [66]:
def create_indexes(connection):
    sql_create_indexes = """
    CREATE INDEX IF NOT EXISTS idx_nyc_311_location ON "nyc_311" USING gist (location);
    CREATE INDEX IF NOT EXISTS idx_nyc_tree_location ON "nyc_tree" USING gist (location);
    CREATE INDEX IF NOT EXISTS idx_nyc_zipcode_location ON "nyc_zipcode" USING gist (location);
    """

    with connection.cursor() as cursor:
        cursor.execute(sql_create_indexes)

In [67]:
engine = create_engine(DB_URL)
create_tables(engine)

In [68]:
with psycopg2.connect(DB_URL) as connection:
    create_indexes(connection)

### Add Data to Database

In [69]:
def write_dataframes_to_table(df_name, table_name):
    engine = create_engine(DB_URL)
    
    # if it cantains geometry data, use "gpd.to_postgis", else use "pd.to_sql"
    try:
        df_name.to_postgis(table_name, engine, if_exists='replace', index=False)
    except:
        df_name.to_sql(table_name, engine, if_exists='replace', index=False)

In [70]:
write_dataframes_to_table(data_311, 'nyc_311')
write_dataframes_to_table(data_tree, 'nyc_tree')
write_dataframes_to_table(data_zipcode,"nyc_zipcode")
write_dataframes_to_table(data_zillow,"nyc_zillow")

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)